In [2]:
########### Python 2.7 #############
import urllib
import requests, json
import lxml.html
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient 
import pandas as pd
import json
import pprint



class IndeedCrawl(object):

    def __init__(self,data_table):
        #self.query = "jobtitle:({0})".format(company_name)
        
        self.endpoint = 'http://api.indeed.com/ads/apisearch'
        self.key = "6437595457989494"
        self.client = MongoClient("mongodb://127.0.0.1:27017")
        self.db=self.client.jobs[data_table]

    def _create_params(self, query, start, location, job_type ,count=3):
        params = urllib.urlencode({
            # Request parameters
            'publisher': self.key,
            'q': query,
            'limit': count,
            'co': "US",
            'start': start,
            'sort': 'date',
            'st': 'employer',
            'format': 'json',
            'v': 2,
            'l': location,
            'filter':1,
            'jt': job_type
        })
        return params

    def search(self, query, start, location, job_type,count=10):
        jobs = []
        try:
            while not start > count:
                params = self._create_params(query, start, location,job_type, count)
                response = requests.get(self.endpoint, params)
                response = json.loads(response.text)
                count = response['totalResults']
                print "start= ", start
                print count

                for res in response['results']:
                    if count == 0:
                        break
                    if res.get('url'):
                        url = res.get('url')
                        res2 = requests.get(url)
                        jd = ""
                        summary = lxml.html.fromstring(res2.text)
                        for span in summary.xpath('//span[@id="job_summary"]'):
                            for x in span.itertext():
                                #job_sum = x.encode('utf-8')
                                job_sum=x
                                jd += job_sum
                        res['job_summary'] = jd
                        self.db.insert(res)
                    print " job: ", res['jobtitle'], "\n"
                    jobs.append(res)
                print "start query again"
                start += 25
                #count -= 25
                
            return jobs
        except Exception as e:
            print e.message
    
    def output_csv(self,filename):

        data = self.db.find({})
        data=list(data)
        data=pd.DataFrame(data)
        data=data.drop_duplicates("jobkey")
        data.to_csv(str(filename)+".csv",encoding="utf-8")
    
    def output_excel(self,filename):

        data = self.db.find({})
        data=list(data)
        data=pd.DataFrame(data)
        data=data.drop_duplicates("jobkey")
        data=data.drop("_id",1)
        data.to_excel(str(filename)+".xlsx",encoding="utf-8")

####################################



In [41]:
search_term="data"
location="95691"
data_table="indeed_sacramento_research"
job_type="parttime" #internship, parttime, fulltime
result_count=500
my_test=IndeedCrawl(data_table)
result = my_test.search(search_term,0,location,job_type,result_count)

start=  0
205


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:66: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


 job:  Tracking/purchase/bidding 

 job:  Student Assistant - Policy and Reports Analyst 

 job:  Student Assistant - Compliance/Title VI 

 job:  Virtual Part-Time Client Services Accountant - California 

 job:  Yard Specialist – Weekend Part Time – 16 Hours 

 job:  Student Assistant - Office Assistant (Sac State Students ONLY) 

 job:  ABA Therapist - 1:1 Tutor 

 job:  RETAIL DATA COLLECTION ASSOCIATE PART TIME 

 job:  RETAIL DATA COLLECTION ASSOCIATE PART TIME 

 job:  Student Assistant - Service Center Help Desk 

 job:  Resident House Manager 

 job:  Student Assistant - CA Department of Parks and Recreation 

 job:  Accounting Specialist 

 job:  (Bilingual spanish) Scheduling Coordinator/Receptionist 

 job:  Clerical Assistant 

 job:  Hydrologic Technician, GS-1316-05 (SRM-DEU) 

 job:  Homeless Outreach Field Team (formerly known as Navigator) 

 job:  Payroll Administrator 

 job:  Registered Behavior Technician 

 job:  Part Time Behavior Technician 

 job:  Behavior Te

 job:  Instructor - Part Time - General Education - Sacramento 

 job:  Instructor - Part Time - Medical Office Specialist - Sacramento 

 job:  Instructor - Part Time - Medical Assistant - Sacramento 

 job:  Registered Behavior Technician 

 job:  Recovery Specialist I - Mental Health 

 job:  Floor Staff 

start query again
start=  175
200
 job:  Complex Workers Compensation Technical Claim Specialist, Commercial Insurance 

 job:  Elementary School Substitute Teacher 

 job:  Homeless Outreach Navigator (Part-Time & Full-Time) 

 job:  Marketing Assistant 

 job:  Entry Level Behavior Techs & RBTs 

 job:  Veterinary Part-time Receptionist 

 job:  LVN/LPT - Mental Health - On Call 

 job:  Telemarketer/Appointment Setter 

 job:  Behavior Technician 

 job:  Retail Delivery Driver Partner (Choose Your Own Hours) – Sacramento 

 job:  Retail Sales Associate 

 job:  Service Center Support PT 

 job:  Accounting Technician 

 job:  Videographer 

 job:  Behavior Technician 

 job:  

In [43]:

my_test.output_excel("output\parttime-sacramento-data")

In [39]:
my_test.db.drop()

In [40]:
my_test.db.count()

0